In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import os
from PIL import Image
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
import cv2 
import math
#
import keras
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras.models import Model
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from keras.utils import multi_gpu_model
#
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.preprocessing import image
import warnings
from os.path import split

In [3]:
# Reference: https://www.kaggle.com/gimunu/data-augmentation-with-keras-into-cnn
# Load images
train_images = os.listdir("/media/fanglab/DATA/Wei-Tse/Whale_Classifier/train/")
test_images = os.listdir("/media/fanglab/DATA/Wei-Tse/Whale_Classifier/test/")
df = pd.read_csv("train.csv")
# Build Table
df["imagepath"] = df["Image"].map( lambda x : "/media/fanglab/DATA/Wei-Tse/Whale_Classifier/train/"+x)
ImageToLabelDict = dict( zip( df.Image, df["Id"]))
df.head()

,Image,Id,imagepath
0,00022e1a.jpg,w_e15442c,/media/fanglab/DATA/Wei-Tse/Whale_Classifier/t...
1,000466c4.jpg,w_1287fbc,/media/fanglab/DATA/Wei-Tse/Whale_Classifier/t...
2,00087b01.jpg,w_da2efe0,/media/fanglab/DATA/Wei-Tse/Whale_Classifier/t...
3,001296d5.jpg,w_19e5482,/media/fanglab/DATA/Wei-Tse/Whale_Classifier/t...
4,0014cfdf.jpg,w_f22f3e3,/media/fanglab/DATA/Wei-Tse/Whale_Classifier/t...


In [4]:
# Build CNN imagedatabase

batch_size = 64
seednum = 1

#generator for training
# the folder name will be though as label name if class_mode="categorical"
# And keras will use folder name for one hot encoding
# let's why alphabery order is important

# Data Augmentation
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True)

#data from specific path
trainD = train_gen.flow_from_directory(
        'CNN_TransferL/train/',
        class_mode="categorical",
        batch_size=batch_size,
        target_size=(224, 224),
        seed=seednum,
        shuffle=False)

Found 9850 images belonging to 4251 classes.


In [5]:
# Got One-hot encoding in keras
encoded_dict=trainD.class_indices
# Inverse encoded : labelname
dict_encodedlabel={a :b for b,a in encoded_dict.items()}

In [8]:
# Load Model
from keras.models import load_model
model = load_model('VGG_TL.h5')
# optimizer 
ot=keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.95, amsgrad=False)
#Compile the model
model.compile(optimizer=ot, loss='categorical_crossentropy', metrics=['accuracy'])

/home/fanglab/.local/lib/python3.6/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [9]:
with open("sample_submission.csv","w") as f:
    with warnings.catch_warnings():
        f.write("Image,Id\n")
        warnings.filterwarnings("ignore",category=DeprecationWarning)
        for imagename in test_images:
            img = image.load_img( 'CNN_TransferL/test/'+imagename+'.png', target_size=(224, 224))
            img =image.img_to_array(img)/255
            #applying preprocessing to test images
            img =img.reshape(1,224,224,3)
            y = model.predict(img)
            # choose top 5 prediction
            predicted_args = np.argsort(y)[0][::-1][:4]
            predicted_tags='new_whale'
            for i in range(0,4):
                predicted_tags=predicted_tags+' '+dict_encodedlabel[predicted_args[i]]
            # image name without '.jpg'
            imagename = split(imagename)[-1]
            f.write("%s,%s\n" %(imagename, predicted_tags))
                                